In [1]:
data_link = '/kaggle/input/auto-correct-dataset/auto_correct.txt'

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import nltk 
import bs4 
import re 
import string 
from collections import Counter 
from nltk.tokenize import word_tokenize 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# let's load our data 
# we have an artical, now we need to get all words in this artical in one list 


def load_words(data_file):
    # create list to get all words 
    words =[]
    with open(data_file ,'r') as file:
        load_file = file.read()
        # ignore all white space 
        process_file = re.findall(r'\w+' , load_file)
        for word in process_file :
            words.append(word.lower())
    return words 


# now let's load data 

data = load_words(data_link)
data[:10]

['this',
 'is',
 'the',
 '100th',
 'etext',
 'file',
 'presented',
 'by',
 'project',
 'gutenberg']

In [4]:
# let's create our vocabular --> remove repeated words

vocab = set(data)
print(f'number of words before: {len(data)}')
print(f'number of words after: {len(vocab)}')
print(f'number of repeated: {len(data) - len(vocab)}')

number of words before: 929396
number of words after: 23902
number of repeated: 905494


<h1 style= "background: #ea97ad; text-align:center">Word Count </h1>

- we need to get counts of words to calculate probabilities of each word 
- let's get an example for what we get from get_count function 

| **Key** |**Value**|
|---------|---------|
| I       | 2       |
| am      | 2       |
|happy    | 1       |
|running  | 1       |


In [5]:
def get_count(word_list):
    final_dict = dict()
    final_dict = Counter(word_list)
    return final_dict 

# let's get it now 


count_words = get_count(data)

In [6]:
print(f'| key\tvalue\t|')
print(f'| ---\t-----\t|')
for i in range(6):
    get_keys =list( count_words.keys())
    print(f'| {get_keys[i]}\t{count_words[f"{get_keys[i]}"]}\t|')
    

| key	value	|
| ---	-----	|
| this	6853	|
| is	9784	|
| the	27660	|
| 100th	1	|
| etext	245	|
| file	20	|


<h1 style= "background: #ea97ad; text-align:center">Get Probability </h1>

In [7]:
# now it's time for get probability to each word 
# now we have dictionary of {words: count }
# and we have length of all words in artical to get probability of each of them.

def get_proba(count_words):
    proba_dict = dict()
    total_words = sum(count_words.values())
    for word , word_value in count_words.items():
        proba_dict[word] = word_value / total_words 
    return proba_dict 


words_proba = get_proba(count_words)

In [8]:
print(f'| word\t|probability\t\t|')
print(f'| ------|-----------------------|')
for i in range(6):
    get_keys =list( words_proba.keys())
    print(f'| {get_keys[i]}\t|{words_proba[f"{get_keys[i]}"]}\t|')
    

| word	|probability		|
| ------|-----------------------|
| this	|0.007373606083951298	|
| is	|0.01052726717136721	|
| the	|0.029761264304989477	|
| 100th	|1.0759676176785783e-06	|
| etext	|0.00026361206633125167	|
| file	|2.1519352353571567e-05	|


<h1 style="background: #ea97ad;text-align:center">Performing String Manipulations</h1>

Now, that we have computed $P(w_i)$ for all the words in the corpus, we will write a few functions to manipulate strings so that we can edit the erroneous strings and return the right spellings of the words. In this section, we will implement four functions: 

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 

In [9]:
# let's start with delete_letter 

def delete_letter(word , verbos = False):
    # to explain what will happen 
    # we just need two lists 
    # 1. split list ---> return a tuple of ( 'c' , 'ans') 
    # 2. delete list ---> return a word deleted one charater each time 
    
    
    split_list  = [(word[:i], word[i:]) for i in range(len(word))]
    delete_list = [L+C[1:] for L,C in split_list]
    
    if verbos:
        print(f'word:{word}\nsplit list: {split_list}\ndelete list: {delete_list}')
    return delete_list 


# let's check our function 

get_items = delete_letter(word='cans' , verbos=True)

word:cans
split list: [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's')]
delete list: ['ans', 'cns', 'cas', 'can']


In [10]:
# second function is switching 

def switching_letter(word , verbos= False):
    
    split_list = [(word[:i],word[i:]) for i in range(len(word))]

    switch_list= [L+R[1]+R[0]+R[2:] for L,R in split_list  if len(R)>=2]
    
    if verbos :
        print(f'word: {word}\nsplit list:{split_list}\nswitching list: {switch_list}')
    return switch_list 

# let's check our function 

get_items = switching_letter(word='ate' , verbos = True)

word: ate
split list:[('', 'ate'), ('a', 'te'), ('at', 'e')]
switching list: ['tae', 'aet']


In [11]:
# replace operation 

def replace(word  , verbos = False):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_list  =[ (word[0:i] , word[i:]) for i in range(len(word))  ]
    replace_list=[ L+ letter+(R[1:] if len(R)>1 else '') for L,R in split_list for letter in letters    ]
    replace_list= sorted( set(replace_list))
    if verbos:
        print(f'word: {word}\nsplit list: {split_list}\nreplace list: {replace_list}')
    return replace_list


# let's test function 
get_items = replace(word='can' , verbos=True)

word: can
split list: [('', 'can'), ('c', 'an'), ('ca', 'n')]
replace list: ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'can', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


In [12]:
# let's do last functio 

def insert(word , verbos =False):
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_list = [(word[:i] , word[i:]) for i in range(len(word)+1)]
    insert_list= [L+ letter+R for L,R in split_list for letter in letters]
    insert_list = sorted( set(insert_list)  )
    if verbos:
        print(f'word:{word}\nsplit list:{split_list}\ninsert list:{insert_list}')
    return insert_list

# let's check 

get_items  = insert(word='at' , verbos=True)


word:at
split list:[('', 'at'), ('a', 't'), ('at', '')]
insert list:['aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat']


<h2 style="background: #ea97ad;text-align:center">Edit One Letter</h2>

In [13]:
def edit_one_letter(word , switch = True):
    
    edit_one_set = set()
    edit_one_set.update(delete_letter(word))
    if switch:
        edit_one_set.update(switching_letter(word))
    edit_one_set.update(replace(word))
    edit_one_set.update(insert(word))
    if word in edit_one_set:
        edit_one_set.remove(word)
    return edit_one_set




In [14]:
# let's check 
word = 'at'
word_edit = edit_one_letter(word=word)
listed = sorted( list(word_edit))


print(f'word:{word}:\nedit one letter: {listed}' )

word:at:
edit one letter: ['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']


In [15]:
# let's do edit to letter 


def edit_two_letter(word , switch=True):
    
    edit_two_set =set()
    edit_one = edit_one_letter(word , switch)
    for W in edit_one:
        edit_two = edit_one_letter(W,switch)
        edit_two_set.update(edit_two)
    return edit_two_set 


In [16]:
# let's check 
word = 'a'
editing = edit_two_letter(word = word)
final_words = sorted( list(editing))
print(f'word:{word}\nEdit Two Letters:\n{final_words}')

word:a
Edit Two Letters:
['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag', 'aah', 'aai', 'aaj', 'aak', 'aal', 'aam', 'aan', 'aao', 'aap', 'aaq', 'aar', 'aas', 'aat', 'aau', 'aav', 'aaw', 'aax', 'aay', 'aaz', 'ab', 'aba', 'abb', 'abc', 'abd', 'abe', 'abf', 'abg', 'abh', 'abi', 'abj', 'abk', 'abl', 'abm', 'abn', 'abo', 'abp', 'abq', 'abr', 'abs', 'abt', 'abu', 'abv', 'abw', 'abx', 'aby', 'abz', 'ac', 'aca', 'acb', 'acc', 'acd', 'ace', 'acf', 'acg', 'ach', 'aci', 'acj', 'ack', 'acl', 'acm', 'acn', 'aco', 'acp', 'acq', 'acr', 'acs', 'act', 'acu', 'acv', 'acw', 'acx', 'acy', 'acz', 'ad', 'ada', 'adb', 'adc', 'add', 'ade', 'adf', 'adg', 'adh', 'adi', 'adj', 'adk', 'adl', 'adm', 'adn', 'ado', 'adp', 'adq', 'adr', 'ads', 'adt', 'adu', 'adv', 'adw', 'adx', 'ady', 'adz', 'ae', 'aea', 'aeb', 'aec', 'aed', 'aee', 'aef', 'aeg', 'aeh', 'aei', 'aej', 'aek', 'ael', 'aem', 'aen', 'aeo', 'aep', 'aeq', 'aer', 'aes', 'aet', 'aeu', 'aev', 'aew', 'aex', 'aey', 'aez', 'af', 'afa', 'afb', 'afc'

<h2 style="background: #ea97ad;text-align:center">Make Suggestions </h2>

In [17]:

def make_suggestions(word, proba ,vocab ,n=2 ,verbos=False):
    suggestions = []
    top_n_suggestions=[]
    
    suggestions = list( (word in vocab and word) or edit_one_letter(word).intersection(vocab) or edit_two_letter(word).intersection(vocab) )
    top_sug = [[s,proba[s]] for s in list(suggestions)  ]
    top_proba = max([proba[s] for s in suggestions])
    top_n_suggestions = top_sug[:n]
    if verbos:
        print(f'word: {word}\nSuggestions:\n{suggestions}')
        print(f'top probability :{top_proba}')
    return top_n_suggestions


In [18]:
# let's show result 
word  = 'dys'
proba = words_proba
vocab = vocab
n     = 2
verbos= True

get_items = make_suggestions(word, proba, vocab ,n,verbos)
print(f'top 2 : \n{get_items}')

word: dys
Suggestions:
['dis', 'dye', 'days', 'dy', 'des']
top probability :0.00022487723209482286
top 2 : 
[['dis', 5.379838088392892e-06], ['dye', 5.379838088392892e-06]]


<h1 style="background: #ea97ad;text-align:center">Minimum Edit Distance</h1>

<h2 style="background: #ea97ad;text-align:center">Dynamic Programming</h2>

Dynamic Programming breaks a problem down into subproblems which can be combined to form the final solution. Here, given a string source[0..i] and a string target[0..j], we will compute all the combinations of substrings[i, j] and calculate their edit distance. To do this efficiently, we will use a table to maintain the previously computed substrings and use those to calculate larger substrings. We can update each element in the matrix.


In [19]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: a string corresponding to the string we are starting with
        target: a string corresponding to the string we want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
    # use deletion and insert cost as  1
    m = len(source) 
    n = len(target) 
    
    #initialize cost matrix with zeros and dimensions (m+1,n+1) 
    D = np.zeros((m+1, n+1), dtype=int) 
    
    # Fill in column 0, from row 1 to row m, both inclusive
    for row in range(1,m+1): 
        D[row,0] = D[row-1, 0]+del_cost
        
    # Fill in row 0, for all columns from 1 to n, both inclusive
    for col in range(1,n+1): 
        D[0,col] = D[0, col-1]+ins_cost
        
    # Loop through row 1 to row m, both inclusive
    for row in range(1,m+1): 
        
        # Loop through column 1 to column n, both inclusive
        for col in range(1,n+1):
            
            # Intialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost
            
            # Check to see if source character at the previous row
            # matches the target character at the previous column, 
            # (source) 'TAY' (target) 'SAY': Check if 'T'=='S'
            if source[row-1] == target[col-1]:
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0
                
            # Update the cost at row, col based on previous entries in the cost matrix
            # Refer to the equation calculate for D[i,j] (the minimum of three calculated costs)

            cost_del = D[row-1, col] + del_cost
            cost_ins = D[row, col-1] + ins_cost
            cost_replace = D[row-1, col-1] + r_cost
            
            D[row,col] = min(cost_del, cost_ins, cost_replace)
          
    # Set the minimum edit distance with the cost found at row m, column n
    med = D[m, n]
    
    return D, med

In [20]:
source =  'eer'
target = 'near'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  3 

   #  n  e  a  r
#  0  1  2  3  4
e  1  2  1  2  3
e  2  3  2  3  4
r  3  4  3  4  3
